In [ ]:

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

# Bulk RNA data - prepare for use in training/testing

In [ ]:
bulk_RNA = pd.read_csv('/Users/alex/Documents/BIOL0041-Project/OAC_masters_project/data/bulk_RNA_classified.csv', header=None)

In [ ]:
bulk_RNA.columns = bulk_RNA.loc[0]

In [ ]:
bulk_RNA = bulk_RNA.drop(0)

In [ ]:
bulk_RNA = bulk_RNA.iloc[:, 2:]

In [ ]:
bulk_RNA = bulk_RNA.dropna()

In [ ]:
bulk_RNA

## Data pre-processing

In [ ]:
bulk_RNA['proliferation'] = bulk_RNA['proliferation'].map({'slow':1,'fast':0})
bulk_RNA['proliferation'].value_counts()


In [ ]:
#Split classifications from training data
X = bulk_RNA.drop('proliferation', axis=1)
y = bulk_RNA['proliferation']

In [ ]:
X = pd.DataFrame(X)
X = X.apply(pd.to_numeric, errors='coerce')


## Removing genes not expressed in at least 45% of dataset

In [ ]:
81/(81+113)

In [ ]:
X.shape

In [ ]:
zero_prop = (X == 0).sum()/len(X)

In [ ]:
zero_prop.sort_values(ascending=False).head(20)

In [ ]:
X = X.loc[:, zero_prop <= 0.45]

## 1000 most variably expressed genes

In [ ]:
var_X = X.var(numeric_only=True)
top_1000_VG = var_X.sort_values(ascending=False).head(1000).index
X = X[top_1000_VG]

## Split data into test/train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

## RF w/ training-testing-predicting

In [ ]:
param_dist = {'bootstrap': [True, False],
              'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'n_estimators': [100, 150, 200, 250, 500, 750, 1000] 
              }

rf = RandomForestClassifier(criterion='gini', random_state= 42)

rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=2000, 
                                 cv=5)

rand_search.fit(X_train, y_train)

In [ ]:
#save best model
best_rf = rand_search.best_estimator_

#print parameters
print(rand_search.best_params_)

In [ ]:
#manual save incase env crashes post-training
#best_rf = RandomForestClassifier(criterion='gini', random_state= 42, n_estimators = 500, min_samples_split = 2, min_samples_leaf = 1, max_features = 'sqrt', max_depth = 10, bootstrap = True)

In [ ]:
best_rf.fit(X_train, y_train)

#get predictions of best model
y_pred = best_rf.predict(X_test)

#create and plot confusion matrix
cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(confusion_matrix=cm).plot();

In [ ]:
y_pred = best_rf.predict(X_test)

probs_rf = best_rf.predict_proba(X_test) [:,1]

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


In [ ]:
#order genes by importance
RF_feature_importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

RF_feature_importances[0:40].plot.bar();

## XGBoost classifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
y_train

In [ ]:
params = {
              'n_estimators': [100, 150, 200, 250, 500, 750, 1000] ,
              'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
              'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'learning_rate':[0.001, 0.01, 0.1, 0.2, 0.3],
              'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
              'gamma': [0, 0.25, 0.5, 1.0],
              'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
              }

XGB = XGBClassifier(random_state = 42)

rand_search_XGB = RandomizedSearchCV(XGB, 
                                 param_distributions = params, 
                                 n_iter=1000, 
                                 cv=5,
                                 scoring = "accuracy")

rand_search_XGB.fit(X_train, y_train)


In [ ]:
#save best model
best_XGB = rand_search_XGB.best_estimator_ 
best_XGB.fit(X_train, y_train)
predictions = best_XGB.predict(X_test)


In [ ]:
#print model params
print(rand_search_XGB.best_params_)

In [ ]:
#manual best params from output above in case reset env
bestXGB = XGBClassifier(random_state = 42, subsample = 0.9, reg_lambda = 1.0, n_estimators = 200, min_child_weight = 0.5, max_depth = 110, learning_rate = 0.1, gamma = 0, colsample_bytree = 0.9, colsample_bylevel = 0.8)
best_XGB.fit(X_train, y_train)
predictions = best_XGB.predict(X_test)

In [ ]:
len(X_test)

In [ ]:
#create and plot confusion matrix
cm = confusion_matrix(y_test, predictions)
ConfusionMatrixDisplay(confusion_matrix=cm).plot();
accuracy = accuracy_score(y_test, predictions)
print(accuracy)


In [ ]:
XGB_feature_importances = pd.Series(best_XGB.feature_importances_, index=X_train.columns).sort_values(ascending=False)

In [ ]:
XGB_feature_importances[0:40].plot.bar();

In [ ]:
gb_disp = RocCurveDisplay.from_estimator(best_XGB, X_test, y_test)

In [ ]:
ax = plt.gca()
rfc_disp = RocCurveDisplay.from_estimator(best_rf, X_test, y_test, ax=ax, alpha=0.8)
gb_disp.plot(ax=ax, alpha=0.8)
plt.show()

## Averaging feature importances

In [ ]:
averaged_importances = {}

for x in RF_feature_importances.index:
    if x in list(XGB_feature_importances.index):

        GB_importance = XGB_feature_importances[x]
        RF_importance = RF_feature_importances[x]
        mean_importance = (GB_importance + RF_importance) / 2
        
        averaged_importances[x] = mean_importance


In [ ]:
ordered_importances = sorted(averaged_importances, key=averaged_importances.get, reverse=True)

In [ ]:
importance_df = pd.DataFrame(ordered_importances)

In [ ]:
importance_df['mean_importance'] = importance_df[0].map(averaged_importances)

In [ ]:
importance_df.rename(columns={0: "gene_name"}, inplace=True)

In [ ]:
importance_df.to_csv("Bulk_feature_importances.csv", sep=",", header = True)

# Single cell RNA data

In [ ]:
import scanpy as sc
from scipy import sparse

In [ ]:
adata = sc.read_h5ad("/Users/alex/Documents/BIOL0041-Project/OAC_masters_project/data/adata_with_rounded.h5ad")

## Get gene list of G0 signature

In [ ]:
upregulated_genes = pd.read_csv('/Users/alex/Documents/BIOL0041-Project/OAC_masters_project/data/upregulated_genes.csv', header=None)
upregulated_genes = upregulated_genes[[1]]
upregulated_genes = list(upregulated_genes[1])

In [ ]:
downregulated_genes = pd.read_csv('/Users/alex/Documents/BIOL0041-Project/OAC_masters_project/data/downregulated_genes.csv', header=None)
downregulated_genes = downregulated_genes[[1]]
downregulated_genes = list(downregulated_genes[1])

In [ ]:
signature_genes = downregulated_genes + upregulated_genes

In [ ]:
#sanity check, should = 139
len(signature_genes)

## Preparing dataset

In [ ]:
# shifted log normalisation is independent cell to cell - can perform train/test splits without having to re-normalise afterwards
adata.X = adata.layers['log1p_norm']

In [ ]:
#sanity check, ~8
adata.X.max()

In [ ]:
adata_obs_df = pd.DataFrame(adata.obs)

In [ ]:
adata_obs_df['cell_type'] = np.where(~adata_obs_df['G0_class'].isna(), adata_obs_df['G0_class'],  adata_obs_df['cell_type'])

In [ ]:
adata_obs_df.cell_type.value_counts()

In [ ]:
adata.obs = adata_obs_df

In [ ]:
ML_adata = adata[adata.obs["cell_type"].isin(['G0 arrested', 'fast cycling'])]
non_Sig_genes = [name for name in ML_adata.var_names if not name in signature_genes]
ML_adata = ML_adata[:, non_Sig_genes]

In [ ]:
ML_adata.obs.cell_type.value_counts()

In [ ]:
ML_adata.X.shape

In [ ]:
sc.pp.filter_genes(ML_adata, min_cells = 150)

In [ ]:
sc.pp.highly_variable_genes(ML_adata, layer = 'log1p_norm', n_top_genes = 1000)

In [ ]:
ML_adata.var.highly_variable.value_counts()

In [ ]:
ML_adata = ML_adata[:, ML_adata.var["highly_variable"]]

In [ ]:
#should have 1000 most variable genes
ML_adata.X.shape

## transforming labels to numbers

In [ ]:
def label_G0(x):
    if x == 'G0 arrested':
        return(1)
    else:
        return(0)

In [ ]:
X = ML_adata.X

In [ ]:
y = ML_adata.obs.cell_type.map(label_G0)

In [ ]:
y.value_counts()

## Train-test splitting data

In [ ]:
#create testing and training datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

## Random forest - SC

In [ ]:
param_dist = {'bootstrap': [True, False],
              'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'n_estimators': [100, 150, 200, 250, 500, 750, 1000] 
              }

rf = RandomForestClassifier(criterion='gini', random_state= 42)

rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=1000, 
                                 cv=5)

rand_search.fit(X_train, y_train)

In [ ]:
#save the best model
best_rf = rand_search.best_estimator_

#output model params
print(rand_search.best_params_)

In [ ]:
#manually set best_RF from above in case reset environment
#best_rf = RandomForestClassifier(criterion='gini', random_state= 42, n_estimators = 250, min_samples_split = 2, min_samples_leaf = 4, max_depth = 50, bootstrap = True,max_features='sqrt')
#best_rf.fit(X_train, y_train)

In [ ]:
#get predictions for best performing model
y_pred = best_rf.predict(X_test)

#create and plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm).plot();

In [ ]:
y_pred = best_rf.predict(X_test)

probs_rf = best_rf.predict_proba(X_test) [:,1]

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


In [ ]:
RF_feature_importances = pd.DataFrame(best_rf.feature_importances_,
             index=ML_adata.var_names).sort_values(0, ascending=False)

In [ ]:
RF_feature_importances

## XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
params = {
              'n_estimators': [100, 150, 200, 250, 500, 750, 1000] ,
              'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
              'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'learning_rate':[0.001, 0.01, 0.1, 0.2, 0.3],
              'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
              'gamma': [0, 0.25, 0.5, 1.0],
              'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
              }

XGB = XGBClassifier(random_state = 42)

rand_search_XGB = RandomizedSearchCV(XGB, 
                                 param_distributions = params, 
                                 n_iter=1000, 
                                 cv=5,
                                 scoring = "accuracy")

rand_search_XGB.fit(X_train, y_train)

In [ ]:
#save the best model
best_XGB = rand_search_XGB.best_estimator_

#output model params
print(rand_search_XGB.best_params_)

In [ ]:
#manual best XGB in case reset env
#best_XGB = XGBClassifier(subsample = 0.6, reg_lambda = 0.1, n_estimators = 100, min_child_weight = 1.0, max_depth = 30, learning_rate = 0.01, gamma = 0, colsample_bytree = 0.6, colsample_bylevel = 0.6, random_state = 42)
#best_XGB.fit(X_train, y_train)

In [ ]:
#get predictions for best performing model
predictions = best_XGB.predict(X_test)

#create and plot confusion matrix
cm = confusion_matrix(y_test, predictions)

ConfusionMatrixDisplay(confusion_matrix=cm).plot();

accuracy = accuracy_score(y_test, predictions)
print(accuracy)

In [ ]:
XGB_feature_importances = pd.DataFrame(best_XGB.feature_importances_,
             index=ML_adata.var_names).sort_values(0, ascending=False)

## SC - averaging feature importances

In [ ]:
#convert the feature importance dfs to dictionaries so can use same averaging method as for the bulk ML data
RF_feature_importances = RF_feature_importances.reset_index().rename(columns={"index":"gene"})	
XGB_feature_importances = XGB_feature_importances.reset_index().rename(columns={"index":"gene"})	

RF_feature_importances = dict(zip(RF_feature_importances['gene'], RF_feature_importances[0]))
XGB_feature_importances = dict(zip(XGB_feature_importances['gene'], XGB_feature_importances[0]))

In [ ]:
averaged_importances = {}

for x in RF_feature_importances.keys():
    if x in list(XGB_feature_importances.keys()):

        XGB_importance = XGB_feature_importances[x]
        RF_importance = RF_feature_importances[x]
        mean_importance = (XGB_importance + RF_importance) / 2
        
        averaged_importances[x] = mean_importance

In [ ]:
ordered_importances = sorted(averaged_importances, key=averaged_importances.get, reverse=True)
ordered_importancesimportance_df = pd.DataFrame(ordered_importances)
importance_df = pd.DataFrame(ordered_importances)
importance_df['mean_importance'] = importance_df[0].map(averaged_importances)

In [ ]:
importance_df = importance_df.rename(columns={0: 'gene_name'})

In [ ]:
importance_df.to_csv("SC_feature_importance.csv", sep=",", index=True, header=True)

In [ ]:
XGB_disp = RocCurveDisplay.from_estimator(best_XGB, X_test, y_test, ax=ax, alpha=0.8)
ax = plt.gca()
rfc_disp = RocCurveDisplay.from_estimator(best_rf, X_test, y_test, ax=ax, alpha=0.8)
gb_disp.plot(ax=ax, alpha=0.8)
plt.show()